# Day 17

In [1]:
from itertools import takewhile
import math

def parse_range(rng):
    _, rng = rng.strip().split('=')
    return [int(n) for n in rng.split('..')]

def parse_target_area(s):
    _, s = s.split(':')
    return [parse_range(rng) for rng in s.strip().split(',')]

def probe(vx, vy):
    x, y = 0, 0
    while True:
        x += vx
        y += vy
        if vx != 0:
            vx -= 1 if vx > 0 else -1
        vy -= 1
        yield x, y
        
def shoot(vx, vy, xrng, yrng):
    g = probe(vx, vy)
    return [*takewhile(
        lambda t: t[0] <= xrng[1] and t[1] >= yrng[0],
        g
    )]
        
def find_all_vx(x_min, x_max):
    if x_min >= 0:
        k = 1
    elif x_max >= 0:
        return 0
    else:
        x_min, x_max = -x_max, -x_min
        k = -1
    g = (math.sqrt(8*x + 1) for x in range(x_min, x_max+1))
    xarr = (int(x) for x in g if not x % 1)
    return (k * (x - 1) // 2 for x in xarr)

minmax = lambda *args: (min(args), max(args))

def check_vy(vy, y_min, y_max):
    height = vy * (vy + 1) // 2
    y_min = height - y_min
    y_max = height - y_max
    y_min, y_max = minmax(y_min, y_max)
    g = (math.sqrt(8*y + 1) for y in range(y_min, y_max+1))
    return height, bool([int(x) for x in g if not x % 1])

def find_vy_and_height(y_min, y_max, height_max=0):
    vy, height = 0, 0
    while True:
        vy_next = vy + 1
        height_next, in_target = check_vy(vy_next, y_min, y_max)
        if not in_target and height_next > height_max:
            return vy, height
        vy = vy_next
        height = height_next
        
def find_max_height(xrng, yrng, *, height_max=0):
    vx = next(find_all_vx(*xrng))
    vy, height = find_vy_and_height(*yrng, height_max=height_max)
    return height, (vx, vy)

def process_and_check(s, *, height_max=0):
    xrng, yrng = parse_target_area(s)
    height, v = find_max_height(xrng, yrng, height_max=height_max)
    path = shoot(v[0], v[1], xrng, yrng)
    assert height == max(y for _, y in path)
    print(f'Maximum height for target_area {[xrng, yrng]} is {height} and is reached with v = {v}')

In [2]:
s = 'target area: x=20..30, y=-10..-5'
process_and_check(s)
with open('day17.txt', 'r') as f:
    process_and_check(f.read(), height_max=12000)

Maximum height for target_area [[20, 30], [-10, -5]] is 45 and is reached with v = (6, 9)
Maximum height for target_area [[88, 125], [-157, -103]] is 12246 and is reached with v = (13, 156)


In [3]:
s = 'target area: x=20..30, y=-10..-5'
xrng, yrng = parse_target_area(s)
print(*find_all_vx(*xrng))

6 7


In [4]:
from itertools import chain, tee
import re
regexp = re.compile(r'\s+')

g = '''
23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
8,-2    27,-8   30,-5   24,-7
'''.strip().split('\n')

g = chain.from_iterable(regexp.split(line) for line in g)
g = (x.split(',') for x in g)
g = ([int(n) for n in t] for t in g)
x_arr, y_arr = tee(g, 2)
x_arr, y_arr = [x for x, y in x_arr], [y for _, y in y_arr]
minmax(*x_arr), minmax(*y_arr)

((6, 30), (-10, 9))

In [5]:
def check_point(p, xrng, yrng):
    x, y = p
    return x >= xrng[0] and y <= yrng[1]

def check_path(path, xrng, yrng):
    return any(check_point(p, xrng, yrng) for p in path)

def analyze(vxrng, vyrng, xrng, yrng):
    d = {
        (vx, vy): shoot(vx, vy, xrng, yrng)
        for vx in range(vxrng[0], vxrng[1] + 1)
        for vy in range(vyrng[0], vyrng[1] + 1)
    }
    d = {
        v: max(y for _, y in path)
        for v, path in d.items()
        if check_path(path, xrng, yrng)
    }
    return d

In [6]:
d = analyze((6, 30), (-10, 9), (20, 30), (-10, -5))
print(len(d))
max(d.values())

112


45

In [7]:
with open('day17.txt', 'r') as f:
    xrng, yrng = parse_target_area(f.read())
xrng, yrng

([88, 125], [-157, -103])

In [8]:
d = analyze((0, 800), (-200, 200), xrng, yrng)
print(len(d))
max(d.values())

3528


12246